In [87]:
"""
MOVING AVERAGE CROSSOVER STRATEGY USING THE S&P 500 AND 5 YEARS OF DATA

First,  we use the backtesting library to find the best crossover strategy using  5 years of data.
Next, we confirm the results using our own backtesting and metrics.

The backtesting library choose short =220, long =230 . It tested combinations using short = 5 to 365 with a step of 5
and long = 5 to 365 with a step of 5. 

This result was similar to the result obtained with 20 years of data.


"""

import numpy as np
import pandas as pd
import hvplot.pandas
from pathlib import Path
import alpaca_trade_api as tradeapi
import requests
import os 
from dotenv import load_dotenv
import plotly.express as px
import panel as pn
import hvplot
import hvplot.pandas

from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA
from datetime import datetime, timedelta


pn.extension()

pd.set_option("display.max_rows", 2000)
pd.set_option("display.max_columns", 2000)
pd.set_option("display.width", 1000)

pd.set_option("display.max_rows", None, "display.max_columns", None)

%matplotlib inline

In [88]:
#load alpaca keys

load_dotenv()

alpaca_api_key = os.getenv("Api_key")
alpaca_secret_key = os.getenv("Secret_key")

api = tradeapi.REST(alpaca_api_key, alpaca_secret_key, api_version='v2')


type(alpaca_api_key)

str

In [89]:
#Use alpha vantage to download 20 years of data for the our selected ticker. 

ticker = 'SPY'

stock_data_df = api.alpha_vantage.historic_quotes(ticker, adjusted=True, output_format='pandas')
stock_data_df.head()

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient
date,,,,,,,,
2020-06-29,301.4100,304.61,298.9300,304.46,304.46,73235506.0,0.0,1.0
2020-06-26,306.1600,306.39,299.4200,300.05,300.05,127811745.0,0.0,1.0
2020-06-25,303.4700,307.64,301.2800,307.35,307.35,88966079.0,0.0,1.0
2020-06-24,309.8400,310.51,302.1000,304.09,304.09,132067392.0,0.0,1.0
2020-06-23,313.4801,314.50,311.6101,312.05,312.05,68066900.0,0.0,1.0


In [90]:
#Clean data

#Sort earliest to latest.
stock_data_df.sort_index(inplace=True, ascending=True)

# Drop nulls
stock_data_df.dropna(inplace=True)

# drop duplicates
stock_data_df.drop_duplicates(inplace=True)

#count nulls 
stock_data_df.isnull().sum()

stock_data_df.head()



,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient
date,,,,,,,,
2000-06-28,145.6250,146.9843,145.3125,145.5625,99.4832,5347700.0,0.0,1.0
2000-06-29,144.7500,145.7500,143.5156,144.1875,98.5435,6345700.0,0.0,1.0
2000-06-30,143.9375,145.5937,143.8906,145.2812,99.2910,7420200.0,0.0,1.0
2000-07-03,145.4375,147.4375,145.1875,147.2812,100.6579,1436600.0,0.0,1.0
2000-07-05,146.3750,146.6562,144.3750,144.6250,98.8425,2748200.0,0.0,1.0


In [91]:
#Set up stock data for backtesting, needs to be OHLCV

stock_data_df.rename(columns={'1. open':'Open','2. high':'High','3. low':'Low', '5. adjusted close':'Close', '6. volume':'Volume'}, inplace=True)
stock_data_df.drop(columns=['4. close','7. dividend amount', '8. split coefficient'], inplace=True)
stock_data_df.sort_index(ascending=True, inplace=True)
stock_data_df.tail()
#stock_data_df.to_csv('stock_data.csv')


,Open,High,Low,Close,Volume
date,,,,,
2020-06-23,313.4801,314.50,311.6101,312.05,68066900.0
2020-06-24,309.8400,310.51,302.1000,304.09,132067392.0
2020-06-25,303.4700,307.64,301.2800,307.35,88966079.0
2020-06-26,306.1600,306.39,299.4200,300.05,127811745.0
2020-06-29,301.4100,304.61,298.9300,304.46,73235506.0


In [92]:
#Slice out 5 years of data
stock_data_df = stock_data_df.iloc[3770:]
stock_data_df.head(10)

,Open,High,Low,Close,Volume
date,,,,,
2015-06-25,211.10,211.25,209.77,189.8563,97107361.0
2015-06-26,210.29,210.58,209.16,189.9196,104174840.0
2015-06-29,208.05,209.83,205.33,185.9113,202621332.0
2015-06-30,207.26,207.32,205.28,186.2913,182925106.0
2015-07-01,207.73,208.03,206.56,187.8114,135979873.0
2015-07-02,208.07,208.27,206.81,187.5852,104373652.0
2015-07-06,205.77,207.65,205.53,186.9971,117975369.0
2015-07-07,206.96,208.17,204.11,188.2095,173820249.0
2015-07-08,206.42,206.76,204.25,185.0608,164020059.0


In [85]:
#Function to do a backtest using Bactesting library. This function will find the best SMA cross strategy.

def backtest():
    class SmaCross(Strategy):
        
        n1=1
        n2=365
        
        def init(self):
            Close = self.data.Close
            self.ma1 = self.I(SMA, Close,self.n1)
            self.ma2 = self.I(SMA, Close,self.n2)

        def next(self):
            if crossover(self.ma1, self.ma2):
                self.buy()
            elif crossover(self.ma2, self.ma1):
                self.sell()

        
                
    bt_stock = Backtest(stock_data_df, SmaCross,
                              cash=10000, commission=.000)
  
    stats = bt_stock.optimize(n1=range(5, 365, 5),
                    n2=range(5, 365, 5),
                    maximize='Equity Final [$]',
                    constraint=lambda p: p.n1 < p.n2)
   

    bt_stock.run()
    #bt_spy_plot = bt_stock.plot()
    #return bt_stock_plot
    return stats
    #return bt_stock.run()
backtest()


C:\Users\costa\Anaconda3\envs\vizenv\lib\site-packages\ipykernel_launcher.py:28: UserWarning:

Searching best of 2556 configurations.



Start                         2015-06-25 00:00:00
End                           2020-06-29 00:00:00
Duration                       1831 days 00:00:00
Exposure [%]                              62.8072
Equity Final [$]                          16994.5
Equity Peak [$]                           18687.6
Return [%]                                69.9448
Buy & Hold Return [%]                     60.3634
Max. Drawdown [%]                        -33.6999
Avg. Drawdown [%]                          -2.213
Max. Drawdown Duration          296 days 00:00:00
Avg. Drawdown Duration           23 days 00:00:00
# Trades                                        6
Win Rate [%]                                   50
Best Trade [%]                            26.7702
Worst Trade [%]                          -2.75352
Avg. Trade [%]                            9.50924
Max. Trade Duration             798 days 00:00:00
Avg. Trade Duration             192 days 00:00:00
Expectancy [%]                            10.6819


In [93]:
# To confirm the above results (short = 220, long = 230) we will do our own tests. 

#To set up the crossover strategy select the one column we need, "Close", and set to dataframe
signals_df = stock_data_df['Close'].to_frame()

signals_df.head()


,Close
date,
2015-06-25,189.8563
2015-06-26,189.9196
2015-06-29,185.9113
2015-06-30,186.2913
2015-07-01,187.8114


In [94]:
# Set the short window and long windows
short_window = 220
long_window = 230

# Set the `date` column as the index
#signals_df.set_index("date", drop=True)

# Generate the short and long moving averages (215 and 225 days, respectively)
signals_df["SMA220"] = signals_df["Close"].rolling(window=short_window).mean()
signals_df["SMA230"] = signals_df["Close"].rolling(window=long_window).mean()
signals_df["Signal"] = 0.0

# Generate the trading signal 0 or 1,
# where 0 is when the short SMA is under the long SMA, and
# where 1 is when the short SMA is higher (or crosses over) the long SMA
signals_df["Signal"][short_window:] = np.where(
    signals_df["SMA220"][short_window:] > signals_df["SMA230"][short_window:], 1.0, 0.0
)

# Calculate the points in time at which a position should be taken, 1 or -1
signals_df["Entry/Exit"] = signals_df["Signal"].diff()

signals_df.head(10)

,Close,SMA220,SMA230,Signal,Entry/Exit
date,,,,,
2015-06-25,189.8563,NaN,NaN,0.0,NaN
2015-06-26,189.9196,NaN,NaN,0.0,0.0
2015-06-29,185.9113,NaN,NaN,0.0,0.0
2015-06-30,186.2913,NaN,NaN,0.0,0.0
2015-07-01,187.8114,NaN,NaN,0.0,0.0
2015-07-02,187.5852,NaN,NaN,0.0,0.0
2015-07-06,186.9971,NaN,NaN,0.0,0.0
2015-07-07,188.2095,NaN,NaN,0.0,0.0
2015-07-08,185.0608,NaN,NaN,0.0,0.0


In [96]:
# Visualize exit position relative to close price
exit = signals_df[signals_df['Entry/Exit'] == -1.0]['Close'].hvplot.scatter(
    color='red',
    marker='v',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize entry position relative to close price
entry = signals_df[signals_df['Entry/Exit'] == 1.0]['Close'].hvplot.scatter(
    color='green',
    marker='^',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize close price for the investment
security_close = signals_df[['Close']].hvplot(
    line_color='lightgray',
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize moving averages
moving_avgs = signals_df[['SMA220', 'SMA230']].hvplot(
    ylabel='Price in $',
    width=1000,
    height=400
)

# Overlay plots
entry_exit_plot = security_close * moving_avgs * entry * exit
entry_exit_plot.opts(xaxis=None)

:Overlay
   .Curve.I          :Curve   [date]   (Close)
   .NdOverlay.I      :NdOverlay   [Variable]
      :Curve   [date]   (value)
   .Scatter.Close.I  :Scatter   [date]   (Close)
   .Scatter.Close.II :Scatter   [date]   (Close)

In [97]:
# Set initial capital
initial_capital = float(100000)

signals_df["Portfolio Cash"] = 100000

# Set the share size
share_size = 1307

# Take a position where the dual moving average crossover is 1 (short SMA is greater than long SMA)
signals_df['Position'] = share_size * signals_df['Signal']

#Find the points in time where a share position is bought or sold
signals_df['Entry/Exit Position'] = signals_df['Position'].diff()

# Multiply share price by entry/exit positions and get the cumulatively sum
signals_df['Portfolio Holdings'] = signals_df['Close'] * signals_df['Entry/Exit Position'].cumsum()

# Subtract the initial capital by the portfolio holdings to get the amount of liquid cash in the portfolio
signals_df['Portfolio Cash'] = signals_df['Portfolio Cash'] - (signals_df['Close'] * signals_df['Entry/Exit Position']).cumsum()

# Get the total portfolio value by adding the cash amount by the portfolio holdings (or investments)
signals_df['Portfolio Total'] = signals_df['Portfolio Cash'] + signals_df['Portfolio Holdings']

# Calculate the portfolio daily returns
signals_df['Portfolio Daily Returns'] = signals_df['Portfolio Total'].pct_change()

# Calculate the cumulative returns
signals_df['Portfolio Cumulative Returns'] = (1 + signals_df['Portfolio Daily Returns']).cumprod() - 1

# # Print the DataFrame
# pd.set_option("display.max_rows", None, "display.max_columns", None)
signals_df.head(50)



,Close,SMA220,SMA230,Signal,Entry/Exit,Portfolio Cash,Position,Entry/Exit Position,Portfolio Holdings,Portfolio Total,Portfolio Daily Returns,Portfolio Cumulative Returns
date,,,,,,,,,,,,
2015-06-25,189.8563,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
2015-06-26,189.9196,NaN,NaN,0.0,0.0,100000.0000,0.0,0.0,0.0000,100000.0000,NaN,NaN
2015-06-29,185.9113,NaN,NaN,0.0,0.0,100000.0000,0.0,0.0,0.0000,100000.0000,0.000000,0.000000
2015-06-30,186.2913,NaN,NaN,0.0,0.0,100000.0000,0.0,0.0,0.0000,100000.0000,0.000000,0.000000
2015-07-01,187.8114,NaN,NaN,0.0,0.0,100000.0000,0.0,0.0,0.0000,100000.0000,0.000000,0.000000
2015-07-02,187.5852,NaN,NaN,0.0,0.0,100000.0000,0.0,0.0,0.0000,100000.0000,0.000000,0.000000
2015-07-06,186.9971,NaN,NaN,0.0,0.0,100000.0000,0.0,0.0,0.0000,100000.0000,0.000000,0.000000
2015-07-07,188.2095,NaN,NaN,0.0,0.0,100000.0000,0.0,0.0,0.0000,100000.0000,0.000000,0.000000
2015-07-08,185.0608,NaN,NaN,0.0,0.0,100000.0000,0.0,0.0,0.0000,100000.0000,0.000000,0.000000


In [98]:
# Visualize exit position relative to total portfolio value
exit = signals_df[signals_df['Entry/Exit'] == -1.0]['Portfolio Total'].hvplot.scatter(
    color='red',
    legend=False,
    ylabel='Total Portfolio Value',
    width=1000,
    height=400
)

# Visualize entry position relative to total portfolio value
entry = signals_df[signals_df['Entry/Exit'] == 1.0]['Portfolio Total'].hvplot.scatter(
    color='green',
    legend=False,
    ylabel='Total Portfolio Value',
    width=1000,
    height=400
)

# Visualize total portoflio value for the investment
total_portfolio_value = signals_df[['Portfolio Total']].hvplot(
    line_color='lightgray',
    ylabel='Total Portfolio Value',
    width=1000,
    height=400
)

# Overlay plots
portfolio_entry_exit_plot = total_portfolio_value * entry * exit
portfolio_entry_exit_plot.opts(xaxis=None)

:Overlay
   .Curve.I                    :Curve   [date]   (Portfolio Total)
   .Scatter.Portfolio_Total.I  :Scatter   [date]   (Portfolio Total)
   .Scatter.Portfolio_Total.II :Scatter   [date]   (Portfolio Total)

In [99]:
#Prepare evaluation metrics dataframe

# Prepare DataFrame for metrics
metrics = [
    'Annual Return',
    'Cumulative Returns',
    'Annual Volatility',
    'Sharpe Ratio',
    'Sortino Ratio']

columns = ['Backtest']

# Initialize the DataFrame with index set to evaluation metrics and column as `Backtest` (just like PyFolio)
portfolio_evaluation_df = pd.DataFrame(index=metrics, columns=columns)
portfolio_evaluation_df

,Backtest
Annual Return,NaN
Cumulative Returns,NaN
Annual Volatility,NaN
Sharpe Ratio,NaN
Sortino Ratio,NaN


In [100]:
# Calculate cumulative return
portfolio_evaluation_df.loc['Cumulative Returns'] = signals_df['Portfolio Cumulative Returns'][-1]
portfolio_evaluation_df



,Backtest
Annual Return,NaN
Cumulative Returns,1.73253
Annual Volatility,NaN
Sharpe Ratio,NaN
Sortino Ratio,NaN


In [101]:
# Calculate annualized return
portfolio_evaluation_df.loc['Annual Return'] = (
    signals_df['Portfolio Daily Returns'].mean() * 252
)
portfolio_evaluation_df

,Backtest
Annual Return,0.240969
Cumulative Returns,1.73253
Annual Volatility,NaN
Sharpe Ratio,NaN
Sortino Ratio,NaN


In [102]:
# Calculate annual volatility
portfolio_evaluation_df.loc['Annual Volatility'] = (
    signals_df['Portfolio Daily Returns'].std() * np.sqrt(252)
)
portfolio_evaluation_df

,Backtest
Annual Return,0.240969
Cumulative Returns,1.73253
Annual Volatility,0.281163
Sharpe Ratio,NaN
Sortino Ratio,NaN


In [103]:
# Calculate Sharpe Ratio
portfolio_evaluation_df.loc['Sharpe Ratio'] = (
    signals_df['Portfolio Daily Returns'].mean() * 252) / (
    signals_df['Portfolio Daily Returns'].std() * np.sqrt(252)
)
portfolio_evaluation_df

,Backtest
Annual Return,0.240969
Cumulative Returns,1.73253
Annual Volatility,0.281163
Sharpe Ratio,0.857042
Sortino Ratio,NaN


In [104]:
# Calculate Downside Return
sortino_ratio_df = signals_df[['Portfolio Daily Returns']].copy()
sortino_ratio_df.loc[:,'Downside Returns'] = 0

target = 0
mask = sortino_ratio_df['Portfolio Daily Returns'] < target
sortino_ratio_df.loc[mask, 'Downside Returns'] = sortino_ratio_df['Portfolio Daily Returns']**2
portfolio_evaluation_df

,Backtest
Annual Return,0.240969
Cumulative Returns,1.73253
Annual Volatility,0.281163
Sharpe Ratio,0.857042
Sortino Ratio,NaN


In [105]:
# Calculate Sortino Ratio
down_stdev = np.sqrt(sortino_ratio_df['Downside Returns'].mean()) * np.sqrt(252)
expected_return = sortino_ratio_df['Portfolio Daily Returns'].mean() * 252
sortino_ratio = expected_return/down_stdev

portfolio_evaluation_df.loc['Sortino Ratio'] = sortino_ratio
portfolio_evaluation_df

,Backtest
Annual Return,0.240969
Cumulative Returns,1.73253
Annual Volatility,0.281163
Sharpe Ratio,0.857042
Sortino Ratio,1.20875


In [106]:
# Initialize trade evaluation DataFrame with columns
trade_evaluation_df = pd.DataFrame(
    columns=[
        'Stock', 
        'Entry Date', 
        'Exit Date', 
        'Shares', 
        'Entry Share Price', 
        'Exit Share Price', 
        'Entry Portfolio Holding', 
        'Exit Portfolio Holding', 
        'Profit/Loss']
)

trade_evaluation_df

,Stock,Entry Date,Exit Date,Shares,Entry Share Price,Exit Share Price,Entry Portfolio Holding,Exit Portfolio Holding,Profit/Loss


In [107]:
# Initialize iterative variables
entry_date = ''
exit_date = ''
entry_portfolio_holding = 0
exit_portfolio_holding = 0
share_size = 0
entry_share_price = 0
exit_share_price = 0

# Loop through signal DataFrame
# If `Entry/Exit` is 1, set entry trade metrics
# Else if `Entry/Exit` is -1, set exit trade metrics and calculate profit,
# Then append the record to the trade evaluation DataFrame
for index, row in signals_df.iterrows():
    if row['Entry/Exit'] == 1:
        entry_date = index
        entry_portfolio_holding = row['Portfolio Holdings']
        share_size = row['Entry/Exit Position']
        entry_share_price = row['Close']

    elif row['Entry/Exit'] == -1:
        exit_date = index
        exit_portfolio_holding = abs(row['Close'] * row['Entry/Exit Position'])
        exit_share_price = row['Close']
        profit_loss = exit_portfolio_holding - entry_portfolio_holding
        trade_evaluation_df = trade_evaluation_df.append(
            {
                'Stock': 'SPY',
                'Entry Date': entry_date,
                'Exit Date': exit_date,
                'Shares': share_size,
                'Entry Share Price': entry_share_price,
                'Exit Share Price': exit_share_price,
                'Entry Portfolio Holding': entry_portfolio_holding,
                'Exit Portfolio Holding': exit_portfolio_holding,
                'Profit/Loss': profit_loss
            },
            ignore_index=True)

# Print the DataFrame
trade_evaluation_df
                

,Stock,Entry Date,Exit Date,Shares,Entry Share Price,Exit Share Price,Entry Portfolio Holding,Exit Portfolio Holding,Profit/Loss
0,SPY,2016-07-11,2016-09-20,1307.0,197.3115,198.3307,257886.1305,259218.2249,1332.0944
1,SPY,2016-09-29,2018-12-06,1307.0,199.5016,260.4992,260748.5912,340472.4544,79723.8632
2,SPY,2018-12-24,2019-06-11,1307.0,227.5772,281.7925,297443.4004,368302.7975,70859.3971


In [109]:
#Create plots

price_df = signals_df[['Close', 'SMA220', 'SMA230']]
price_chart = price_df.hvplot.line()
price_chart.opts(xaxis=None)

:NdOverlay   [Variable]
   :Curve   [date]   (value)

In [110]:
portfolio_evaluation_df.reset_index(inplace=True)
portfolio_evaluation_table = portfolio_evaluation_df.hvplot.table()
portfolio_evaluation_table

:Table   [index,Backtest]

In [111]:
trade_evaluation_table = trade_evaluation_df.hvplot.table()
trade_evaluation_table

:Table   [Stock,Entry Date,Exit Date,Shares,Entry Share Price,Exit Share Price,Entry Portfolio Holding,Exit Portfolio Holding,Profit/Loss]

In [112]:
#Build dashboard with panel

# Create rows
price_chart_row = pn.Row(price_chart)
portfolio_evaluation_row = pn.Row(portfolio_evaluation_table)
trade_evaluation_row = pn.Row(trade_evaluation_table)

# Create columns
portfolio_column = pn.Column('# Portfolio Evaluation Metrics', price_chart_row, portfolio_evaluation_row)
trade_column = pn.Column('# Trade Evaluation Metrics', trade_evaluation_row)

# Create tabs
trading_dashboard = pn.Tabs(
    ("Portfolio Metrics", portfolio_column),
    ("Trade Metrics", trade_column)
)

In [113]:
trading_dashboard.servable()

Tabs
    [0] Column
        [0] Markdown(str)
        [1] Row
            [0] HoloViews(NdOverlay)
        [2] Row
            [0] HoloViews(Table)
    [1] Column
        [0] Markdown(str)
        [1] Row
            [0] HoloViews(Table)